# Homework 1 - Image Classification

The notebook is divided into several sections:
- Setup - Importing libraries, defining the create_csv function, mounting Drive and unzipping the dataset in the proper Drive directory. Indeed, the notebook was created using the Drive integration with Colab, therefore the main directory is the folder /AN2DL/ImageClassification, which was created in advance with the dataset in it.
- Preparing the data - The training set and the validation set are prepared, preprocessing images and creating the Datasets objects to be used by the models.
- Models:
  - First Model*
  - Second Model*
  - Third Model (VGG-16)*
  - Fourth Model (InceptionV3)*
  - Fifth Model (ResNet)*
- Ensemble Method (Mode of the results) - In this last section, the predictions of the last three models are taken to compute the most predicted class for each test sample.

*In each model section, the architecture is defined, the optimization parameters are set, the callbacks are created, the model is trained and finally the predictions on the test set are computed, exporting the results in a csv format.

# Setup

In [1]:
# Importing the necessary libraries and setting the seed(s) to make the code replicable
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [2]:
# Defining the create_csv function, which will be used to export the prediction results on the test set
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
# Mounting Drive to Colab, as the Drive folder /AN2DL/ImageClassification is the main directory for this homework
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzipping the dataset (named "MaskDataset.zip"), which has to be previously put in the homework directory
!unzip '/content/drive/My Drive/AN2DL/ImageClassification/MaskDataset.zip'

# Saving the directories for the dataset, the training set and the test set (to be used later)
cwd = os.getcwd()                                                               # This is the current working directory, in which the dataset has been unzipped
dataset_dir = os.path.join(cwd, 'MaskDataset')                                  # This is the dataset directory, which contains the training and the test folders, along with the json
training_dir = os.path.join(dataset_dir, 'training')                            # This is the training directory, which contains the training samples
test_dir = os.path.join(dataset_dir, 'test')                                    # This is the test directory, which contains the test samples

# Preparing the data

In [5]:
# Creating the ImageDataGenerator objects for Training and Validation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training - Data Augmentation is applied to generate more images for testing the model(s). Additionally, the standard image rescaling is applied.
train_data_gen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0)

train_data_gen = ImageDataGenerator(rescale=1./255)

# Validation - The standard image rescaling is applied.
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Setting the batch size, the image shape and creating the Training and Validation generators

# Batch size 
bs = 2

# Image shape
img_h = 512
img_w = 512

# Building a dataframe containing all the images in the training folder and their targets from the json
import json

with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  targets = json.load(f)                                                        # Loading the target values from the json in a Python dictionary

train_dataframe = pd.DataFrame(targets.items())                                 # Converting the items of the dictionary into a Pandas dataframe
train_dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)     # Renaming the columns of the dataframe
train_dataframe["class"]=train_dataframe["class"].astype(str)                   # Converting the class numbers into strings, to be compatible with the "categorial" class_mode of the generators
np.random.shuffle(train_dataframe.values)                                       # Shuffling the dataframe values

valid_dataframe = train_dataframe[4490:]                                        # Taking the last (5614-4490)=1124 images for the validation set (20% of the samples)...
train_dataframe = train_dataframe [:4490]                                       # ... and the other 4490 images for the training set

# Training Generator
train_gen = train_data_gen.flow_from_dataframe(train_dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)
# Validation Generator
valid_gen = valid_data_gen.flow_from_dataframe(valid_dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

In [7]:
# Creating the Training and Validation datasets from the previous generators, to be used by the model(s)

num_classes = 3                                                                 # The number of classes that we want to classify: 0 (no person wears mask), 1 (all people wear masks), 2 (someone wears mask)

# Training Dataset
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Validation Dataset
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

# First Model

In [ ]:
# This is the same architecture seen in the practical session of the course, but slightly more deeper (depth is 7 instead of 5).
# The starting number of filters is the same (8) and it is doubled after every block.
# Each block is composed by: a convolutional layer with kernel size 3x3 and stride 1; a ReLU activation function; a MaxPooling layer with pool size 2x2.
# Finally, the FC part is composed by the standard Flatten layer, an intermediate layer with 512 units and ReLU activation function, and a SoftMax layer for the output.

start_f = 8
depth = 7

model_1 = tf.keras.Sequential()

# Features Extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model_1.add(tf.keras.layers.Conv2D(filters=start_f, 
                                       kernel_size=(3, 3),
                                       strides=(1, 1),
                                       padding='same',
                                       input_shape=input_shape))
    model_1.add(tf.keras.layers.ReLU())
    model_1.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Classifier
model_1.add(tf.keras.layers.Flatten())
model_1.add(tf.keras.layers.Dense(units=512, activation='relu'))
model_1.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model_1.summary()

# Visualize initialized weights
model_1.weights

In [ ]:
# Optimization parameters
# These are the standard ones, using a Categorical CrossEntropy as loss function and the Adam optimizer with a learning rate of 1e-4.

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# Learning rate and optimizer
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model_1.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Setting up the callbacks and Early Stopping
# The purpose of this piece of code is to create a "classification_experiments" folder inside the directory of this homework (if not already created).
# Inside it, it creates a folder called "Model_1_" followed by the date and the time of execution, to recognize the experiment.
# Then, it sets up the callback for the training of the model, saving the model after each epoch inside the previously mentioned folder, only if the model improved in accuracy on the Validation set.
# Finally, Ealy Stopping is also inserted in the callback, to monitor the loss on the Validation set and to stop the training procedure if it becomes worse for "patience" steps.

# Creating the "classification_experiments" folder if not already created
exps_dir = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
now = datetime.now().strftime('%b%d_%H-%M-%S')

# Creating the folder in which the model will be saved
model_name = 'Model_1'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
# Setting up the callback to save the model after each epoch only if there is an improvement in term of validation accuracy
callbacks = []

ckpt_dir = os.path.join(exp_dir)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exp_dir), 
                                                   monitor='val_accuracy',
                                                   mode='max',
                                                   verbose=0,
                                                   save_best_only=True,         # It saves the model only if the validation accuracy improves
                                                   save_weights_only=False)
callbacks.append(ckpt_callback)

# Early Stopping is inserted in the callback, stopping the training procedure if the validation loss increases for too long
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
# Fitting the model
# The training is done on the train_dataset, while the validation is done on the valid_dataset prepared before.
# It can go on up to 100 epochs, but the Early Stopping callback explained before allows to stop much earlier.

model_1.fit(x=train_dataset,
            epochs=100,  
            steps_per_epoch=len(train_gen),
            validation_data=valid_dataset,
            validation_steps=len(valid_gen), 
            callbacks=callbacks)

In [ ]:
# Testing the model on the given Test set

# If we want to load an already trained model, then we can set load_model to True, otherwise the test will be done on the model trained in the same session
load_model = False
if load_model:
  path = 'Model_1_XXXXXXXXXX'
  full_path = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/classification_experiments', path)
  test_model_1 = tf.keras.models.load_model(full_path)
else:
  test_model_1 = model_1

# Each image from the test set is prepared and fed to the model for the classification. The output dictionary contains, for each test image, the softmax predictions for each class
from PIL import Image

image_filenames = next(os.walk(test_dir))[2]                                    # We collect the filenames of the images in the Test set
predictions_1 = {}

for image_filename in image_filenames:                                          # For every image in the Test set
  img = Image.open(os.path.join(test_dir, image_filename)).convert('RGB')       # We open it in an Image variable
  img = img.resize((img_h, img_w))                                              # We resize it according to the image shape on which the model is trained
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)
  prediction = test_model_1.predict(img_array)                                  # We fed it to the model, obtaining the SoftMax probabilities of that image to belong to the three classes
  predictions_1[image_filename] = prediction                                    # And we put the result in the dictionary, as the value of the image selected

# Then, for each test image, the class with the maximum output value is taken as the predicted class
import ntpath

results_1 = {}

for i in range(0, len(predictions_1)):                                          # For every item in the dictionary created above
  image_name = ntpath.basename(image_filenames[i])                              # We get the name of the image, to use it later
  pred_class = np.argmax(predictions_1[image_name])                             # We retrieve the class (0, 1 or 2) which has the highest SoftMax probability
  results_1[image_name] = str(pred_class)                                       # And we put the class number in the dictionary, as the value of the image selected

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results_1, '/content/drive/My Drive/AN2DL/ImageClassification/')

# Second Model

In [ ]:
# This is the same architecture seen in the practical session of the course, but with a different number of filters.
# Indeed, the initial number of filters is set to 5, and after each block they are quadruplicated instead of doubled.
# Each block is composed by: a convolutional layer with kernel size 3x3 and stride 1; a ReLU activation function; a MaxPooling layer with pool size 2x2.
# Finally, the FC part is composed by the standard Flatten layer, an intermediate layer with 512 units and ReLU activation function, and a SoftMax layer for the output.

start_f = 5
depth = 5

model_2 = tf.keras.Sequential()

# Features Extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model_2.add(tf.keras.layers.Conv2D(filters=start_f,                  
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model_2.add(tf.keras.layers.ReLU())
    model_2.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 4

# Classifier
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(units=512, activation='relu'))
model_2.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model_2.summary()

# Visualize initialized weights
model_2.weights

In [ ]:
# Optimization parameters
# These are the standard ones, using a Categorical CrossEntropy as loss function and the Adam optimizer with a learning rate of 1e-4.

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# Learning rate and optimizer
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model_2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Setting up the callbacks and Early Stopping
# The purpose of this piece of code is to create a "classification_experiments" folder inside the directory of this homework (if not already created).
# Inside it, it creates a folder called "Model_2_" followed by the date and the time of execution, to recognize the experiment.
# Then, it sets up the callback for the training of the model, saving the model after each epoch inside the previously mentioned folder, only if the model improved in accuracy on the Validation set.
# Finally, Ealy Stopping is also inserted in the callback, to monitor the loss on the Validation set and to stop the training procedure if it becomes worse for "patience" steps.

# Creating the "classification_experiments" folder if not already created
exps_dir = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
now = datetime.now().strftime('%b%d_%H-%M-%S')

# Creating the folder in which the model will be saved
model_name = 'Model_2'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
# Setting up the callback to save the model after each epoch only if there is an improvement in term of validation accuracy
callbacks_2 = []

ckpt_dir = os.path.join(exp_dir)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exp_dir), 
                                                   monitor='val_accuracy',
                                                   mode='max',
                                                   verbose=0,
                                                   save_best_only=True,         # It saves the model only if the validation accuracy improves
                                                   save_weights_only=False)
callbacks_2.append(ckpt_callback)

# Early Stopping is inserted in the callback, stopping the training procedure if the validation loss increases for too long
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks_2.append(es_callback)

In [ ]:
# Fitting the model
# The training is done on the train_dataset, while the validation is done on the valid_dataset prepared before.
# It can go on up to 100 epochs, but the Early Stopping callback explained before allows to stop much earlier.

model_2.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),     
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks_2)

In [ ]:
# Testing the model on the given Test set

# If we want to load an already trained model, then we can set load_model to True, otherwise the test will be done on the model trained in the same session
load_model = False
if load_model:
  path = 'Model_2_XXXXXXXXXX'
  full_path = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/classification_experiments', path)
  test_model_2 = tf.keras.models.load_model(full_path)
else:
  test_model_2 = model_2

# Each image from the test set is prepared and fed to the model for the classification. The output dictionary contains, for each test image, the softmax predictions for each class
from PIL import Image

image_filenames = next(os.walk(test_dir))[2]                                    # We collect the filenames of the images in the Test set
predictions_2 = {}

for image_filename in image_filenames:                                          # For every image in the Test set
  img = Image.open(os.path.join(test_dir, image_filename)).convert('RGB')       # We open it in an Image variable
  img = img.resize((img_h, img_w))                                              # We resize it according to the image shape on which the model is trained
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)
  prediction = test_model_2.predict(img_array)                                  # We fed it to the model, obtaining the SoftMax probabilities of that image to belong to the three classes
  predictions_2[image_filename] = prediction                                    # And we put the result in the dictionary, as the value of the image selected

# Then, for each test image, the class with the maximum output value is taken as the predicted class
import ntpath

results_2 = {}

for i in range(0, len(predictions_2)):                                          # For every item in the dictionary created above
  image_name = ntpath.basename(image_filenames[i])                              # We get the name of the image, to use it later
  pred_class = np.argmax(predictions_2[image_name])                             # We retrieve the class (0, 1 or 2) which has the highest SoftMax probability
  results_2[image_name] = str(pred_class)                                       # And we put the class number in the dictionary, as the value of the image selected

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results_2, '/content/drive/My Drive/AN2DL/ImageClassification/')

# Third Model (VGG-16)

In [ ]:
# This model is realized using Transfer Learning with the VGG-16 architecture.
# It is imported through the Keras Applications API, using the ImageNet weights and without including the FC part.
# Indeed, the FC part is composed by a Flatten layer, two Dense layers (each one with 4096 units) to mimic the original VGG-16 architecture, and a SoftMax layer for the output.
# Fine Tuning is applied to the pre-trained network, freezing the imported weight until the 15th layer and training the remaining ones.

# Loading the VGG-16 architecture
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# Setting Fine Tuning
finetuning = True

if finetuning:
    freeze_until = 15 
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

# Creating the whole model, adding the FC part after the loaded architecture    
model_3 = tf.keras.Sequential()
model_3.add(vgg)
model_3.add(tf.keras.layers.Flatten())
model_3.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model_3.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model_3.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model_3.summary()

# Visualize initialized weights
model_3.weights

In [9]:
# Optimization parameters
# Since VGG-16 contains a huge number of parameters, here the Stochastic Gradient Descent (SGD) optimizer has been used to avoid being stuck in local minima.
# The Nesterov Accelerated Gradient variant has been used to improve the optimizer performance.
# The learning rate has been tuned (by trial and error) to a quite small value: 1e-3.

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate and optimizer
lr = 1e-3
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, nesterov=True)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model_3.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
# Setting up the callbacks and Early Stopping
# The purpose of this piece of code is to create a "classification_experiments" folder inside the directory of this homework (if not already created).
# Inside it, it creates a folder called "Model_3_" followed by the date and the time of execution, to recognize the experiment.
# Then, it sets up the callback for the training of the model, saving the model after each epoch inside the previously mentioned folder, only if the model improved in accuracy on the Validation set.
# Finally, Ealy Stopping is also inserted in the callback, to monitor the loss on the Validation set and to stop the training procedure if it becomes worse for "patience" steps.

# Creating the "classification_experiments" folder if not already created
exps_dir = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
now = datetime.now().strftime('%b%d_%H-%M-%S')

# Creating the folder in which the model will be saved
model_name = 'Model_3'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
# Setting up the callback to save the model after each epoch only if there is an improvement in term of validation accuracy
callbacks_3 = []

ckpt_dir = os.path.join(exp_dir)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exp_dir), 
                                                   monitor='val_accuracy',
                                                   mode='max',
                                                   verbose=0,
                                                   save_best_only=True,         # It saves the model only if the validation accuracy improves
                                                   save_weights_only=False)
callbacks_3.append(ckpt_callback)

# Early Stopping is inserted in the callback, stopping the training procedure if the validation loss increases for too long
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks_3.append(es_callback)

In [ ]:
# Fitting the model
# The training is done on the train_dataset, while the validation is done on the valid_dataset prepared before.
# It can go on up to 100 epochs, but the Early Stopping callback explained before allows to stop much earlier.

model_3.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks_3)

In [12]:
# Testing the model on the given Test set

# If we want to load an already trained model, then we can set load_model to True, otherwise the test will be done on the model trained in the same session
load_model = False
if load_model:
  path = 'Model_3_XXXXXXXXXX'
  full_path = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/classification_experiments', path)
  test_model_3 = tf.keras.models.load_model(full_path)
else:
  test_model_3 = model_3

# Each image from the test set is prepared and fed to the model for the classification. The output dictionary contains, for each test image, the softmax predictions for each class
from PIL import Image

image_filenames = next(os.walk(test_dir))[2]                                            # We collect the filenames of the images in the Test set
predictions_3 = {}

for image_filename in image_filenames:                                                  # For every image in the Test set
  img = Image.open(os.path.join(test_dir, image_filename)).convert('RGB')               # We open it in an Image variable
  img = img.resize((img_h, img_w))                                                      # We resize it according to the image shape on which the model is trained
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)
  img_array = tf.keras.applications.vgg16.preprocess_input(img_array, data_format=None) # We preprocess the image for the specific architecture
  prediction = test_model_3.predict(img_array)                                          # We fed it to the model, obtaining the SoftMax probabilities of that image to belong to the three classes
  predictions_3[image_filename] = prediction                                            # And we put the result in the dictionary, as the value of the image selected

# Then, for each test image, the class with the maximum output value is taken as the predicted class
import ntpath

results_3 = {}

for i in range(0, len(predictions_3)):                                          # For every item in the dictionary created above
  image_name = ntpath.basename(image_filenames[i])                              # We get the name of the image, to use it later
  pred_class = np.argmax(predictions_3[image_name])                             # We retrieve the class (0, 1 or 2) which has the highest SoftMax probability
  results_3[image_name] = str(pred_class)                                       # And we put the class number in the dictionary, as the value of the image selected

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results_3, '/content/drive/My Drive/AN2DL/ImageClassification/')

# Fourth Model (InceptionV3)

In [ ]:
# This model is realized using Transfer Learning with the InceptionV3 architecture.
# It is imported through the Keras Applications API, using the ImageNet weights and without including the FC part.
# Indeed, the FC part is composed by a Flatten layer, one Dense layer with 512 units, and a SoftMax layer for the output.
# Fine Tuning is applied to the pre-trained network, freezing the imported weight until the 13th layer and training the remaining ones.

# Loading the InceptionV3 architecture
inception = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# Setting Fine Tuning
finetuning = True

if finetuning:
    freeze_until = 13
    for layer in inception.layers[:freeze_until]:
        layer.trainable = False
else:
    inception.trainable = False

# Creating the whole model, adding the FC part after the loaded architecture
model_4 = tf.keras.Sequential()
model_4.add(inception)
model_4.add(tf.keras.layers.Flatten())
model_4.add(tf.keras.layers.Dense(units=512, activation='relu'))
model_4.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model_4.summary()

# Visualize initialized weights
model_4.weights

In [16]:
# Optimization parameters
# These are the standard ones, using a Categorical CrossEntropy as loss function and the Adam optimizer with a learning rate of 1e-4.

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# Learning rate and optimizer
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model_4.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [17]:
# Setting up the callbacks and Early Stopping
# The purpose of this piece of code is to create a "classification_experiments" folder inside the directory of this homework (if not already created).
# Inside it, it creates a folder called "Model_4_" followed by the date and the time of execution, to recognize the experiment.
# Then, it sets up the callback for the training of the model, saving the model after each epoch inside the previously mentioned folder, only if the model improved in accuracy on the Validation set.
# Finally, Ealy Stopping is also inserted in the callback, to monitor the loss on the Validation set and to stop the training procedure if it becomes worse for "patience" steps.

# Creating the "classification_experiments" folder if not already created
exps_dir = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
now = datetime.now().strftime('%b%d_%H-%M-%S')

# Creating the folder in which the model will be saved
model_name = 'Model_4'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
# Setting up the callback to save the model after each epoch only if there is an improvement in term of validation accuracy
callbacks_4 = []

ckpt_dir = os.path.join(exp_dir)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exp_dir),
                                                   monitor='val_accuracy', 
                                                   mode='max',
                                                   verbose=0,
                                                   save_best_only=True,         # It saves the model only if the validation accuracy improves
                                                   save_weights_only=False)
callbacks_4.append(ckpt_callback)

# Early Stopping is inserted in the callback, stopping the training procedure if the validation loss increases for too long
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks_4.append(es_callback)

In [ ]:
# Fitting the model
# The training is done on the train_dataset, while the validation is done on the valid_dataset prepared before.
# It can go on up to 100 epochs, but the Early Stopping callback explained before allows to stop much earlier.

model_4.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks_4)

In [13]:
# Testing the model on the given Test set

# If we want to load an already trained model, then we can set load_model to True, otherwise the test will be done on the model trained in the same session
load_model = False
if load_model:
  path = 'Model_4_XXXXXXXXXX'
  full_path = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/classification_experiments', path)
  test_model_4 = tf.keras.models.load_model(full_path)
else:
  test_model_4 = model_4

# Each image from the test set is prepared and fed to the model for the classification. The output dictionary contains, for each test image, the softmax predictions for each class
from PIL import Image

image_filenames = next(os.walk(test_dir))[2]                                                    # We collect the filenames of the images in the Test set
predictions_4 = {}

for image_filename in image_filenames:                                                          # For every image in the Test set
  img = Image.open(os.path.join(test_dir, image_filename)).convert('RGB')                       # We open it in an Image variable
  img = img.resize((img_h, img_w))                                                              # We resize it according to the image shape on which the model is trained
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)
  img_array = tf.keras.applications.inception_v3.preprocess_input(img_array, data_format=None)  # We preprocess the image for the specific architecture
  prediction = test_model_4.predict(img_array)                                                  # We fed it to the model, obtaining the SoftMax probabilities of that image to belong to the three classes
  predictions_4[image_filename] = prediction                                                    # And we put the result in the dictionary, as the value of the image selected

# Then, for each test image, the class with the maximum output value is taken as the predicted class
import ntpath

results_4 = {}

for i in range(0, len(predictions_4)):                                          # For every item in the dictionary created above
  image_name = ntpath.basename(image_filenames[i])                              # We get the name of the image, to use it later
  pred_class = np.argmax(predictions_4[image_name])                             # We retrieve the class (0, 1 or 2) which has the highest SoftMax probability
  results_4[image_name] = str(pred_class)                                       # And we put the class number in the dictionary, as the value of the image selected

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results_4, '/content/drive/My Drive/AN2DL/ImageClassification/')

# Fifth Model (ResNet)

In [ ]:
# This model is realized using Transfer Learning with the ResNet50V2 architecture.
# It is imported through the Keras Applications API, using the ImageNet weights and without including the FC part.
# Indeed, the FC part is composed by a Flatten layer, one Dense layer with 512 units, and a SoftMax layer for the output.
# Fine Tuning is applied to the pre-trained network, freezing the imported weight until the 13th layer and training the remaining ones.

# Loading the InceptionV3 architecture
resnet = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# Setting Fine Tuning
finetuning = True

if finetuning:
    freeze_until = 13
    for layer in resnet.layers[:freeze_until]:
        layer.trainable = False
else:
    resnet.trainable = False

# Creating the whole model, adding the FC part after the loaded architecture
model_5 = tf.keras.Sequential()
model_5.add(resnet)
model_5.add(tf.keras.layers.Flatten())
model_5.add(tf.keras.layers.Dense(units=512, activation='relu'))
model_5.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model_5.summary()

# Visualize initialized weights
model_5.weights

In [11]:
# Optimization parameters
# These are the standard ones, using a Categorical CrossEntropy as loss function and the Adam optimizer with a learning rate of 1e-4.

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# Learning rate and optimizer
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model_5.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [12]:
# Setting up the callbacks and Early Stopping
# The purpose of this piece of code is to create a "classification_experiments" folder inside the directory of this homework (if not already created).
# Inside it, it creates a folder called "Model_5_" followed by the date and the time of execution, to recognize the experiment.
# Then, it sets up the callback for the training of the model, saving the model after each epoch inside the previously mentioned folder, only if the model improved in accuracy on the Validation set.
# Finally, Ealy Stopping is also inserted in the callback, to monitor the loss on the Validation set and to stop the training procedure if it becomes worse for "patience" steps.

# Creating the "classification_experiments" folder if not already created
exps_dir = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    
now = datetime.now().strftime('%b%d_%H-%M-%S')

# Creating the folder in which the model will be saved
model_name = 'Model_5'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
# Setting up the callback to save the model after each epoch only if there is an improvement in term of validation accuracy
callbacks_5 = []

if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exp_dir), 
                                                   monitor='val_accuracy',
                                                   mode='max',
                                                   verbose=0,
                                                   save_best_only=True,         # It saves the model only if the validation accuracy improves
                                                   save_weights_only=False)
callbacks_5.append(ckpt_callback)

# Early Stopping is inserted in the callback, stopping the training procedure if the validation loss increases for too long
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks_5.append(es_callback)

In [ ]:
# Fitting the model
# The training is done on the train_dataset, while the validation is done on the valid_dataset prepared before.
# It can go on up to 100 epochs, but the Early Stopping callback explained before allows to stop much earlier.
model_5.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks_5)

In [14]:
# Testing the model on the given Test set

# If we want to load an already trained model, then we can set load_model to True, otherwise the test will be done on the model trained in the same session
load_model = False
if load_model:
  path = 'Model_5_XXXXXXXXXX'
  full_path = os.path.join('/content/drive/My Drive/AN2DL/ImageClassification/classification_experiments', path)
  test_model_5 = tf.keras.models.load_model(full_path)
else:
  test_model_5 = model_5

# Each image from the test set is prepared and fed to the model for the classification. The output dictionary contains, for each test image, the softmax predictions for each class
from PIL import Image

image_filenames = next(os.walk(test_dir))[2]                                                # We collect the filenames of the images in the Test set
predictions_5 = {}

for image_filename in image_filenames:                                                      # For every image in the Test set
  img = Image.open(os.path.join(test_dir, image_filename)).convert('RGB')                   # We open it in an Image variable
  img = img.resize((img_h, img_w))                                                          # We resize it according to the image shape on which the model is trained
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)
  img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array, data_format=None) # We preprocess the image for the specific architecture
  prediction = test_model_5.predict(img_array)                                              # We fed it to the model, obtaining the SoftMax probabilities of that image to belong to the three classes
  predictions_5[image_filename] = prediction                                                # And we put the result in the dictionary, as the value of the image selected

# Then, for each test image, the class with the maximum output value is taken as the predicted class
import ntpath

results_5 = {}

for i in range(0, len(predictions_5)):                                          # For every item in the dictionary created above
  image_name = ntpath.basename(image_filenames[i])                              # We get the name of the image, to use it later
  pred_class = np.argmax(predictions_5[image_name])                             # We retrieve the class (0, 1 or 2) which has the highest SoftMax probability
  results_5[image_name] = str(pred_class)                                       # And we put the class number in the dictionary, as the value of the image selected

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results_5, '/content/drive/My Drive/AN2DL/ImageClassification/')

# Ensemble Method (Mode of the results)

In [15]:
# Here, the predictions of the models on the Test set are put together to perform a majority voting.
# Given their higher accuracies, this method considers only the predictions made by the last three models (the ones created using a Transfer Learning approach).
# The dictionaries containing the predicted class for each image of the Test set are taken as input of the ensemble_majority_voting function.
# For each image, the function computes the mode between the predicted classes, taking the class which has been predicted by more models.

from scipy import stats

# Defining the function for the mode computation
def ensemble_majority_voting(results_3, results_4, results_5):
  modes = {}
  keys = results_3.keys()                                                       # We just take number of keys (images) from one, since the Test set is the same in all of them
  for k in keys:                                                                # For each image in the dictionaries
    array = np.array([results_3[k], results_4[k], results_5[k]])                # The predicted classes are condensed into an array
    modes[k] = stats.mode(array)                                                # And the mode between them is taken
    
  # At this point, the modes have been computed and saved as the values of the corresponding images.
  # However, the "stats.mode(array)" call, returns a "Scipy.Mode" object, and not the single number corresponding to the mode.
  # To obtain a clean dictionary composed by the tuples (test image, most predicted class), we need to take that single number.
  # This is done by taking, for each image, the first element of the first element of the "Scipy.Mode" object "results" for each image.
  results_mv = {}
  keys2 = modes.keys()
  for i in keys2:
    results_mv[i] = np.array(modes[i][0][0]).tolist()

  # Finally, the clean dictionary is returned by the function.
  return results_mv

In [16]:
# The function is called, taking as input the dictionaries containing the predictions made by the last three models.

# Calling the function on the actual results returned by the models
results = ensemble_majority_voting(results_3, results_4, results_5)

# Finally, the results (test image, predicted class) are exported in a csv format
create_csv(results, '/content/drive/My Drive/AN2DL/ImageClassification/')